In [31]:
import torch
import torch.nn.functional as F

from model import MLP
from vocab import get_vocab

In [126]:
TEXT_FILE = "./data/names.txt"
MODEL_FILE = "./models/1704743300/epoch_103.ckpt"
EOT_TOKEN = "<|endoftext|>"
LATENT_SIZE = 8
HIDDEN_SIZE = 64
NUM_LAYERS = 2

In [127]:
with open(TEXT_FILE, "r") as f:
    text_data = [line.strip() for line in f.readlines()]

vocab_dict, vocab_size = get_vocab(text_data, EOT_TOKEN)

In [128]:
model = MLP(input_size=1,
            latent_size=LATENT_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            output_size=vocab_size).to("cuda")

In [129]:
model.load_state_dict(torch.load(MODEL_FILE))

<All keys matched successfully>

In [130]:
parameters = sum(p.numel() for p in model.parameters())
parameters

6891

In [143]:
temperature = 1.0

input = 0
name = ""
latent = torch.randn(LATENT_SIZE).to("cuda")

while True:
    tensor_input = torch.tensor(input, dtype=torch.float32).to("cuda").view(-1, 1)
    
    output = model(
        x=tensor_input,
        latent=latent
    )
    output = F.softmax(output / temperature, dim=-1)
    index = output.multinomial(num_samples=1, replacement=True).item()
    if index == vocab_size - 1:
        break
    letter = vocab_dict["id2letter"][index]
    name += letter
    input += 1

name

'toaa'